In [8]:
import findspark 
findspark.init("/Users/jean/spark-2.4.4-bin-hadoop2.7")
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

In [2]:
spark = SparkSession.builder.appName('recsys').getOrCreate()

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
data = spark.read.csv("movielens_ratings.csv", inferSchema=True, header=True)

In [5]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
train_data, test_data = data.randomSplit([0.7,0.3])

In [9]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")

In [10]:
model = als.fit(train_data)

In [11]:
prediction = model.transform(test_data)

In [12]:
prediction.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    26|  0.8431685|
|     31|   1.0|    27| -3.4813275|
|     31|   1.0|    13|  0.3394712|
|     31|   1.0|    19|  1.1376143|
|     31|   3.0|     8|  1.9105952|
|     31|   1.0|     0|  1.4211707|
|     85|   1.0|    28|  1.7256306|
|     85|   1.0|    13|-0.01903484|
|     85|   5.0|    16| 0.50233155|
|     85|   1.0|     4|   1.799432|
|     65|   1.0|    28| -2.7756054|
|     65|   1.0|    19|  1.1238214|
|     53|   5.0|     8|  1.0710009|
|     53|   1.0|    23| -0.5088533|
|     53|   3.0|    14|  3.4633458|
|     78|   1.0|    27| 0.28498948|
|     78|   1.0|    12|  1.0237032|
|     78|   1.0|     1|  0.9945226|
|     78|   1.0|    13| 0.21689893|
|     78|   1.0|    19| 0.84613264|
+-------+------+------+-----------+
only showing top 20 rows



In [13]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [14]:
evaluator.evaluate(prediction)

2.590405355813104

In [16]:
single_user = test_data.filter(test_data["userId"]==1).select(['movieId', 'userId'])

In [17]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|     1|
|     37|     1|
|     40|     1|
|     43|     1|
|     62|     1|
|     76|     1|
|     78|     1|
|     86|     1|
|     88|     1|
+-------+------+



In [18]:
recommendations = model.transform(single_user)

In [19]:
recommendations.orderBy("prediction", ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     62|     1|  2.464555|
|     40|     1|  1.676818|
|     88|     1| 1.6732032|
|      6|     1|  1.477823|
|     37|     1| 1.4230615|
|     43|     1|  1.141915|
|     86|     1| 1.0751338|
|     78|     1| 0.9945226|
|     76|     1|-0.0421454|
+-------+------+----------+

